# Seq2Seq Translation with LSTM  
### English to French Machine Translation

**Bao gồm:**
- Chuẩn bị dữ liệu (raw → processed)
- Tạo từ vựng (src + tgt)
- Dataset + DataLoader
- Encoder–Decoder LSTM
- Training loop (teacher forcing)
- Inference (translate function)
- Evaluation (BLEU score)
- 5 ví dụ dịch + phân tích lỗi

---

### Import thư viện 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from nltk.translate.bleu_score import sentence_bleu
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
import random


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\MSI VN\.conda\envs\nlp310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\MSI VN\.conda\envs\nlp310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\MSI VN\.conda\envs\nlp310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\MSI VN\.conda\envs\nlp310\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  Fil

In [4]:
import sys
print(sys.executable)

c:\Users\MSI VN\.conda\envs\nlp310\python.exe


# 1. Load dataset

In [5]:
# def load_parallel(en_path, fr_path):
#     en_lines = open(en_path, 'r', encoding='utf-8').read().strip().split('\n')
#     fr_lines = open(fr_path, 'r', encoding='utf-8').read().strip().split('\n')

#     assert len(en_lines) == len(fr_lines), "Số dòng 2 file không khớp nhau!"

#     pairs = []

#     for en, fr in zip(en_lines, fr_lines):
#         pairs.append((en.lower(), fr.lower()))
    
#     return pairs

# train_pairs = load_parallel('./data/raw/train.en', './data/raw/train.fr')
# val_pairs = load_parallel('./data/raw/val.en', './data/raw/val.fr')


# 2. Build Vocabulary

In [6]:
# SPECIL_TOKENS = ['<pad>', '<sos>', '<eos>', '<unk>']

# def tokenize(sentence):
#     return sentence.strip().split()

# def yield_tokens(pairs):
#     for en, fr in pairs:
#         for token in tokenize(en):
#             yield token
#         for token in tokenize(fr):
#             yield token

# def build_vocab(iterator):
#     vocab = build_vocab_from_iterator(iterator, 
#                                     min_freq=2,
#                                     specials=SPECIL_TOKENS,
#                                     special_first=True,
#                                     max_tokens=10000)
#     vocab.set_default_index(vocab['<unk>'])
#     return vocab

# from torchtext.vocab import build_vocab_from_iterator



In [7]:
# # === Cell: Chuẩn bị dữ liệu (tokenize, build vocab, dataset, dataloader) ===
# # Yêu cầu: raw files dạng data/raw/train.en, data/raw/train.fr, valid.en/valid.fr, test.en/test.fr
# # Sử dụng spaCy tokenizer; vocab giới hạn 10000; thêm tokens: <unk>, <pad>, <sos>, <eos>.
# # Nếu torchtext khả dụng -> dùng build_vocab_from_iterator; nếu không -> fallback bằng Counter.

# import warnings
# from collections import Counter
# from torch.nn.utils.rnn import pad_sequence

# # --- spaCy tokenizer (bắt buộc đã cài en_core_web_sm và fr_core_news_sm) ---
# import spacy
# en_nlp = spacy.load("en_core_web_sm")
# fr_nlp = spacy.load("fr_core_news_sm")

# def tokenize_en(text):
#     return [tok.text.lower() for tok in en_nlp(text)]

# def tokenize_fr(text):
#     return [tok.text.lower() for tok in fr_nlp(text)]

# # --- Try to import torchtext build_vocab_from_iterator; fallback if not present ---
# USE_TORCHTEXT = False
# try:
#     from torchtext.vocab import build_vocab_from_iterator
#     USE_TORCHTEXT = True
# except Exception as e:
#     warnings.warn("torchtext unavailable or failed to load; falling back to Counter-based vocab builder.\n" + str(e))
#     USE_TORCHTEXT = False

# # --- Special tokens and options ---
# PAD_TOKEN = "<pad>"
# SOS_TOKEN = "<sos>"
# EOS_TOKEN = "<eos>"
# UNK_TOKEN = "<unk>"
# SPECIALS = [UNK_TOKEN, PAD_TOKEN, SOS_TOKEN, EOS_TOKEN]
# MAX_VOCAB = 10000
# MIN_FREQ = 1   # you can set to 2 to filter very rare tokens if desired

# # --- Helpers to read parallel files (EN/FR) ---
# def load_parallel(en_path, fr_path):
#     with open(en_path, encoding="utf8") as f:
#         en_lines = [l.strip() for l in f if l.strip()]
#     with open(fr_path, encoding="utf8") as f:
#         fr_lines = [l.strip() for l in f if l.strip()]
#     assert len(en_lines) == len(fr_lines), f"Mismatch lines: {len(en_lines)} != {len(fr_lines)}"
#     return list(zip(en_lines, fr_lines))

# # Example usage:
# # train_pairs = load_parallel("data/raw/train.en", "data/raw/train.fr")

# # --- Build vocab: two implementations ---
# if USE_TORCHTEXT:
#     # Using torchtext.build_vocab_from_iterator (preferred when available)
#     def yield_tokens_from_pairs(pairs, lang="en"):
#         if lang == "en":
#             for en, fr in pairs:
#                 yield tokenize_en(en)
#         else:
#             for en, fr in pairs:
#                 yield tokenize_fr(fr)

#     def build_vocab_torchtext(pairs, lang="en", max_tokens=MAX_VOCAB, min_freq=MIN_FREQ):
#         iterator = yield_tokens_from_pairs(pairs, lang=lang)
#         vocab = build_vocab_from_iterator(iterator,
#                                           specials=SPECIALS,
#                                           special_first=True,
#                                           max_tokens=max_tokens)
#         vocab.set_default_index(vocab[UNK_TOKEN])
#         return vocab

# else:
#     # Fallback: use Counter and build dict
#     def build_vocab_counter(pairs, lang="en", max_tokens=MAX_VOCAB, min_freq=MIN_FREQ):
#         counter = Counter()
#         if lang == "en":
#             for en, fr in pairs:
#                 counter.update(tokenize_en(en))
#         else:
#             for en, fr in pairs:
#                 counter.update(tokenize_fr(fr))
#         # keep most common up to max_tokens - len(SPECIALS)
#         most_common = [w for w, c in counter.most_common(max_tokens - len(SPECIALS)) if c >= min_freq]
#         vocab_list = SPECIALS + most_common
#         stoi = {w: i for i, w in enumerate(vocab_list)}
#         # helper object to mimic torchtext vocab API minimally
#         class VocabLike:
#             def __init__(self, stoi, itos):
#                 self.stoi = stoi
#                 self.itos = itos
#             def __len__(self):
#                 return len(self.itos)
#             def __getitem__(self, token):
#                 return self.stoi.get(token, self.stoi[UNK_TOKEN])
#             def get_itos(self):
#                 return self.itos
#             def get_stoi(self):
#                 return self.stoi
#             def set_default_index(self, idx):
#                 pass
#         itos = vocab_list
#         return VocabLike(stoi, itos)

# # --- Encode / numericalize helpers (works for both vocab types) ---
# def tokens_to_ids(tokens, vocab, specials_map=None):
#     # vocab could be torchtext.Vocab or our VocabLike
#     ids = []
#     if USE_TORCHTEXT:
#         for t in tokens:
#             ids.append(vocab[t])
#     else:
#         for t in tokens:
#             ids.append(vocab.stoi.get(t, vocab.stoi[UNK_TOKEN]))
#     return ids

# def encode_sentence_en(text, vocab):
#     toks = tokenize_en(text)
#     ids = [ (vocab[SOS_TOKEN] if USE_TORCHTEXT else vocab.stoi[SOS_TOKEN]) ]
#     ids += tokens_to_ids(toks, vocab)
#     ids += [ (vocab[EOS_TOKEN] if USE_TORCHTEXT else vocab.stoi[EOS_TOKEN]) ]
#     return torch.tensor(ids, dtype=torch.long)

# def encode_sentence_fr(text, vocab):
#     toks = tokenize_fr(text)
#     ids = [ (vocab[SOS_TOKEN] if USE_TORCHTEXT else vocab.stoi[SOS_TOKEN]) ]
#     ids += tokens_to_ids(toks, vocab)
#     ids += [ (vocab[EOS_TOKEN] if USE_TORCHTEXT else vocab.stoi[EOS_TOKEN]) ]
#     return torch.tensor(ids, dtype=torch.long)

# # --- Dataset + collate_fn (pad + sort by length descending + return lengths) ---
# class ParallelDataset(Dataset):
#     def __init__(self, pairs, src_vocab, tgt_vocab):
#         self.pairs = pairs
#         self.src_vocab = src_vocab
#         self.tgt_vocab = tgt_vocab

#     def __len__(self):
#         return len(self.pairs)

#     def __getitem__(self, idx):
#         en, fr = self.pairs[idx]
#         src_ids = encode_sentence_en(en, self.src_vocab)
#         tgt_ids = encode_sentence_fr(fr, self.tgt_vocab)
#         return src_ids, tgt_ids

# def collate_fn(batch):
#     # batch: list of (src_tensor, tgt_tensor)
#     src_list, tgt_list = zip(*batch)
#     # sort by src length desc (required for pack_padded_sequence)
#     sorted_pairs = sorted(zip(src_list, tgt_list), key=lambda x: -x[0].size(0))
#     src_list, tgt_list = zip(*sorted_pairs)
#     src_padded = pad_sequence(src_list, batch_first=True, padding_value=(vocab_en[PAD_TOKEN] if USE_TORCHTEXT else vocab_en.stoi[PAD_TOKEN]))
#     tgt_padded = pad_sequence(tgt_list, batch_first=True, padding_value=(vocab_fr[PAD_TOKEN] if USE_TORCHTEXT else vocab_fr.stoi[PAD_TOKEN]))
#     src_lengths = torch.tensor([s.size(0) for s in src_list], dtype=torch.long)
#     tgt_lengths = torch.tensor([t.size(0) for t in tgt_list], dtype=torch.long)
#     return src_padded, tgt_padded, src_lengths, tgt_lengths

# # === Usage example ===
# # load data
# train_pairs = load_parallel("data/raw/train.en", "data/raw/train.fr")
# valid_pairs = load_parallel("data/raw/val.en", "data/raw/val.fr")
# test_pairs  = load_parallel("data/raw/test_2016_flickr.en",  "data/raw/test_2016_flickr.fr")

# # build vocabs (prefer torchtext if available)
# if USE_TORCHTEXT:
#     vocab_en = build_vocab_torchtext(train_pairs, lang="en", max_tokens=MAX_VOCAB, min_freq=MIN_FREQ)
#     vocab_fr = build_vocab_torchtext(train_pairs, lang="fr", max_tokens=MAX_VOCAB, min_freq=MIN_FREQ)
# else:
#     vocab_en = build_vocab_counter(train_pairs, lang="en", max_tokens=MAX_VOCAB, min_freq=MIN_FREQ)
#     vocab_fr = build_vocab_counter(train_pairs, lang="fr", max_tokens=MAX_VOCAB, min_freq=MIN_FREQ)

# print("Vocab sizes -> EN:", len(vocab_en) if USE_TORCHTEXT else len(vocab_en.itos),
#       " FR:", len(vocab_fr) if USE_TORCHTEXT else len(vocab_fr.itos))

# # create dataloaders
# batch_size = 64
# train_ds = ParallelDataset(train_pairs, vocab_en, vocab_fr)
# valid_ds = ParallelDataset(valid_pairs, vocab_en, vocab_fr)
# test_ds  = ParallelDataset(test_pairs,  vocab_en, vocab_fr)

# train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
# valid_loader = DataLoader(valid_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
# test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# print('DataLoaders ready. Example batch shapes:')
# for src_batch, tgt_batch, src_lens, tgt_lens in train_loader:
#     print(src_batch.shape, tgt_batch.shape, src_lens.shape, tgt_lens.shape)
#     break


In [8]:
from processed_data import (build_vocab, load_parallel, ParallelDataset, collate_fn, encode_sentence_en, encode_sentence_fr, save_vocab, load_vocab, save_dataset_pytorch, load_dataset_pytorch, create_loader)

In [9]:
train_pairs = load_parallel('./data/raw/train.en', './data/raw/train.fr')
vocab_en = build_vocab(train_pairs, lang="en")
vocab_fr = build_vocab(train_pairs, lang="fr")

train_ds = ParallelDataset(train_pairs, vocab_en, vocab_fr)



In [11]:
save_vocab(vocab_en, './data/processed/vocab_en.pkl')
save_vocab(vocab_fr, './data/processed/vocab_fr.pkl')

In [12]:
valid_pairs = load_parallel('./data/raw/val.en', './data/raw/val.fr')
test_pairs = load_parallel('./data/raw/test_2016_flickr.en', './data/raw/test_2016_flickr.fr')

valid_ds = ParallelDataset(valid_pairs, vocab_en, vocab_fr)
test_ds = ParallelDataset(test_pairs, vocab_en, vocab_fr)

save_dataset_pytorch(train_ds, './data/processed/train_dataset.pt')
save_dataset_pytorch(valid_ds, './data/processed/valid_dataset.pt')
save_dataset_pytorch(test_ds, './data/processed/test_dataset.pt')